In [11]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

congressional_voting_records = fetch_ucirepo(id=105)

X = congressional_voting_records.data.features
y = congressional_voting_records.data.targets
df = congressional_voting_records.data.original

default_table = {'republican':[0,0,0],'democrat':[0,0,0]}
default_df = pd.DataFrame(data=default_table, index=['y', 'n', '?'])
variables = congressional_voting_records.variables['name']

In [18]:
variables

0                                      Class
1                        handicapped-infants
2                 water-project-cost-sharing
3          adoption-of-the-budget-resolution
4                       physician-fee-freeze
5                            el-salvador-aid
6                religious-groups-in-schools
7                    anti-satellite-test-ban
8                  aid-to-nicaraguan-contras
9                                 mx-missile
10                               immigration
11              synfuels-corporation-cutback
12                        education-spending
13                    superfund-right-to-sue
14                                     crime
15                         duty-free-exports
16    export-administration-act-south-africa
Name: name, dtype: object

In [19]:
def naiveBayesClassifier(learning, validate):
    bayestables = {k:default_df.copy() for k in variables}
    print(bayestables)

naiveBayesClassifier(0,0)

{'Class':    republican  democrat
y           0         0
n           0         0
?           0         0, 'handicapped-infants':    republican  democrat
y           0         0
n           0         0
?           0         0, 'water-project-cost-sharing':    republican  democrat
y           0         0
n           0         0
?           0         0, 'adoption-of-the-budget-resolution':    republican  democrat
y           0         0
n           0         0
?           0         0, 'physician-fee-freeze':    republican  democrat
y           0         0
n           0         0
?           0         0, 'el-salvador-aid':    republican  democrat
y           0         0
n           0         0
?           0         0, 'religious-groups-in-schools':    republican  democrat
y           0         0
n           0         0
?           0         0, 'anti-satellite-test-ban':    republican  democrat
y           0         0
n           0         0
?           0         0, 'aid-to-nicaraguan-cont

In [ ]:
def simpleSample(data):
    testing = data.groupby('class', group_keys=False).sample(frac=0.2)
    learning = data.drop(testing.index)
    return learning, testing

In [ ]:
def fold10Sample(data):
    folds = []
    for i in range(0,10):
        folds.append(data.groupby('class', group_keys=False).sample(4))
        data.drop(folds[i].index)
    return folds

In [ ]:
import numpy as np

m = input("Mode?")
m = int(m)
if m == 1:
    df = transform(df)
    
learning, testing = simpleSample(df)
_ , validate = simpleSample(learning)

success = 0
for v in validate.values:
    #print(learning.drop('class', axis=1), learning['class'])
    ys = naiveBayesClassifier(learning, validate)
    if ys == v[-1]:
        success += 1

print("1.Train Set Accuracy:\nAccuracy:", "{:.2%}".format(success/24))

print("\n2.10-Fold Cross-Validation Results:")
folds = fold10Sample(learning)
successes = [0] * 10
for i in range(0,10):
    learning = df.drop(folds[i].index)
    for t in folds[i].values:
        ys = naiveBayesClassifier(learning, validate)
        if ys == t[-1]:
            successes[i] += 1
    successes[i] = successes[i]/12
    print("Accuracy Fold", i+1, ":", "{:.2%}".format(successes[i]))

print("\nAverage Accuracy:",  "{:.2%}".format(np.average(successes)))
print("Standard Deviation:",  "{:.2%}".format(np.std(successes)))

success = 0
for t in testing.values:
    #print(learning.drop('class', axis=1), learning['class'])
    ys = naiveBayesClassifier(learning, validate)
    if ys == t[-1]:
        success += 1

print("\n3.Test Set Accuracy:\nAccuracy:", "{:.2%}".format(success/30))
